In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [63]:
from langchain_core.tools import BaseTool
from tavily import TavilyClient
import os
import json

class TavilyToolFlexible(BaseTool):
    name: str = "tavily_search_results_json"
    description: str = "Query Tavily and return raw content for LLM to handle."

    def _run(self, query: str) -> str:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        results = client.search(query=query, max_results=1)

        try:
            content = results["results"][0]["content"]
            # Nếu content là JSON lồng dạng chuỗi → parse để dễ đọc
            try:
                parsed = json.loads(content)
                return json.dumps(parsed, indent=2)  # đẹp hơn, dễ đọc với LLM
            except json.JSONDecodeError:
                return content  # content không phải JSON → trả nguyên
        except Exception as e:
            return f"Error while parsing Tavily response: {e}"

    def _arun(self, query: str):
        raise NotImplementedError

tools = [TavilyToolFlexible()]

In [77]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [
    TavilySearchResults(max_results=3,
    api_key=os.getenv("TAVILY_API_KEY"))]

from langgraph.prebuilt import ToolNode

tool_executor = ToolNode(tools)



In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(model_name="llama3-70b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [80]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

In [81]:
from langchain_core.agents import AgentAction

import json
from langchain_core.messages import FunctionMessage


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    # Hỗ trợ cả GPT 3.5/4 (function_call) và GPT-4o/Groq (tool_calls)
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    elif "tool_calls" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"
    
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages" : [response]}



def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    tool_call = last_message.additional_kwargs["tool_calls"][0]  # lấy call đầu tiên
    tool_name = tool_call["function"]["name"]
    tool_args = json.loads(tool_call["function"]["arguments"])

    action = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="",  # để trống nếu không cần
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=tool_name)
    return {"messages": [function_message]}



In [82]:
def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    tool_call = last_message.additional_kwargs["tool_calls"][0]  # lấy call đầu tiên
    tool_name = tool_call["function"]["name"]
    tool_args = json.loads(tool_call["function"]["arguments"])

    action = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="",  # để trống nếu không cần
    )
    response = input(prompt=f"[y/n] continue with: {action}?")
    if response == "n":
        raise ValueError
    

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=tool_name)
    return {"messages": [function_message]}

In [83]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    }
)
workflow.add_edge("action", "agent")

app = workflow.compile()

In [84]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node '{key}':")
        print("------------------")
        print(value)
    print("\n---\n")  

output from node 'agent':
------------------
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_c4xa', 'function': {'arguments': '{"query":"current weather in san francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 945, 'total_tokens': 997, 'completion_time': 0.166850143, 'prompt_time': 0.032704355, 'queue_time': 0.22790129100000003, 'total_time': 0.199554498}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d7ee8c96-2491-4b28-a76a-dd2d5b30a1e6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san francisco'}, 'id': 'call_c4xa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 945, 'output_tokens': 52, 'total_tokens': 997})]}

---

output from node 'action':
------------------
{'messages': [FunctionMessage(content="{'messages

In [70]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is the weather in london?")]}
app.invoke(inputs)


{'messages': [HumanMessage(content='What is the weather in london?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e2yf', 'function': {'arguments': '{"query":"weather in london"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 917, 'total_tokens': 966, 'completion_time': 0.161035886, 'prompt_time': 0.029220172, 'queue_time': 0.22545678400000002, 'total_time': 0.190256058}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b966da00-0db9-45b2-b4c6-dd56028bdafb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in london'}, 'id': 'call_e2yf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 917, 'output_tokens': 49, 'total_tokens': 966}),
  FunctionMessage(content="{'messages': []}", additional_kwargs={}, respons

In [ ]:
from tavily import TavilyClient
import os
from dotenv import load_dotenv

load_dotenv()  # Tải API key từ file .env
os.environ["LANGCHAIN_TRACING_V2"] = "true"
client = TavilyClient()

# Gửi truy vấn tìm kiếm
result = client.search(query="London weather forecast", max_results=1)

# In kết quả
print(result)


In [72]:
from tavily import TavilyClient
client = TavilyClient()
result = client.search(query="London weather forecast", max_results=1)
print(result)

{'query': 'London weather forecast', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in London', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1746352007, 'localtime': '2025-05-04 10:46'}, 'current': {'last_updated_epoch': 1746351900, 'last_updated': '2025-05-04 10:45', 'temp_c': 11.3, 'temp_f': 52.3, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.4, 'wind_kph': 18.4, 'wind_degree': 20, 'wind_dir': 'NNE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 58, 'cloud': 50, 'feelslike_c': 9.2, 'feelslike_f': 48.5, 'windchill_c': 9.8, 'windchill_f': 49.6, 'heatindex_c': 11.8, 'heatindex_f': 53.2, 'dewpoint_c': 1.1, 'dewpoint_f': 34

# LangGraph: Dynamically Returning a Tool Output Directly

In [89]:
from pydantic import BaseModel, Field

class SearchTool(BaseModel):
    query: str = Field(description="query to look up online")
    return_direct:bool = Field(
        default=False,
        description="whether or the result of this should be returned directly to the user without you seeing what it is"

    )

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Tải API key từ file .env

os.environ["LANGCHAIN_TRACING_V2"] = "true"

from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = [
    TavilySearchResults(max_results=3,
    api_key=os.getenv("TAVILY_API_KEY"),
    args_schema=SearchTool)]

tools = search_tool
from langgraph.prebuilt import ToolNode
tool_executor = ToolNode(tools)



In [93]:
from langchain_groq import ChatGroq

model = ChatGroq(model_name="llama3-70b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

In [100]:
from langchain_core.agents import AgentAction

import json
from langchain_core.messages import FunctionMessage


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

  
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    else:
        arguments = json.loads(last_message.additional_kwargs["tool_calls"][0]["function"]["arguments"])
        if arguments.get("return_direct", False):
            return "final"
        else:
            return "continue"
    
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages" : [response]}



def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    tool_call = last_message.additional_kwargs["tool_calls"][0]  # lấy call đầu tiên
    tool_name = tool_call["function"]["name"]
    tool_args = json.loads(tool_call["function"]["arguments"])

    if tool_name == "tavily_search_results_json":
        # Nếu là công cụ tìm kiếm, không cần gọi lại
        if "return_direct" in tool_args:
            del tool_args["return_direct"]
    



    action = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="",  # để trống nếu không cần
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=tool_name)
    return {"messages": [function_message]}



In [101]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.add_node("final", call_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "final": "final",
        "end": END
    }
)
workflow.add_edge("action", "agent")
workflow.add_edge("final", END)

app = workflow.compile()

In [102]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node '{key}':")
        print("------------------")
        print(value)
    print("\n---\n")  

output from node 'agent':
------------------
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cz1j', 'function': {'arguments': '{"query":"current weather in san francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 993, 'total_tokens': 1045, 'completion_time': 0.155212681, 'prompt_time': 0.04202971, 'queue_time': 0.247091823, 'total_time': 0.197242391}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5914505d-e8f1-4247-ad44-fb9d00500883-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san francisco'}, 'id': 'call_cz1j', 'type': 'tool_call'}], usage_metadata={'input_tokens': 993, 'output_tokens': 52, 'total_tokens': 1045})]}

---

output from node 'action':
------------------
{'messages': [FunctionMessage(content="{'messages': []}"

In [103]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf? return this result directly by setting return_direct=True")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node '{key}':")
        print("------------------")
        print(value)
    print("\n---\n")  

output from node 'agent':
------------------
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vd67', 'function': {'arguments': '{"query":"current weather in san francisco","return_direct":true}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1003, 'total_tokens': 1084, 'completion_time': 0.231428571, 'prompt_time': 0.039899472, 'queue_time': 0.249101001, 'total_time': 0.271328043}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ca53507d-5e45-4b13-911c-ff1df2b00b08-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san francisco', 'return_direct': True}, 'id': 'call_vd67', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1003, 'output_tokens': 81, 'total_tokens': 1084})]}

---

output from node 'final':
------------------
{'messages

## LangGraph: Respond in a Specific Format


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()  # Tải API key từ file .env
from pydantic import BaseModel, Field

os.environ["LANGCHAIN_TRACING_V2"] = "true"

from langchain_community.tools.tavily_search import TavilySearchResults
tools = [
    TavilySearchResults(max_results=3)]
from langgraph.prebuilt import ToolNode
tool_executor = ToolNode(tools)

from langchain_groq import ChatGroq

model = ChatGroq(model_name="llama3-70b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )

from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

class Response(BaseModel):
    temperature: float = Field(description="the temperature")
    other_notes: str = Field(description="any other notes about the weather")


functions = [format_tool_to_openai_function(t) for t in tools]
functions.append(convert_pydantic_to_openai_function(Response))
model = model.bind_functions(functions)

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

from langchain_core.agents import AgentAction
import json
from langchain_core.messages import FunctionMessage


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

  
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    elif last_message.additional_kwargs["tool_calls"][0]["function"]["name"] == "Response":
        return "end"
    else:
        return "continue"

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages" : [response]}



def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    tool_call = last_message.additional_kwargs["tool_calls"][0]  # lấy call đầu tiên
    print(last_message.additional_kwargs["tool_calls"])
    tool_name = tool_call["function"]["name"]
    tool_args = json.loads(tool_call["function"]["arguments"])

    action = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="",  # để trống nếu không cần
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=tool_name)
    return {"messages": [function_message]}

from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)


workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",

        "end": END
    }
)
workflow.add_edge("action", "agent")


app = workflow.compile()

from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node '{key}':")
        print("------------------")
        print(value)
    print("\n---\n")  

output from node 'agent':
------------------
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xgt3', 'function': {'arguments': '{"temperature":0,"other_notes":"Please provide the current temperature"}', 'name': 'Response'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 1075, 'total_tokens': 1129, 'completion_time': 0.208917822, 'prompt_time': 0.035049803, 'queue_time': 0.278181148, 'total_time': 0.243967625}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6c436fee-5c2c-4f7a-a3bf-13c0596dfc3e-0', tool_calls=[{'name': 'Response', 'args': {'temperature': 0, 'other_notes': 'Please provide the current temperature'}, 'id': 'call_xgt3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1075, 'output_tokens': 54, 'total_tokens': 1129})]}

---



## Add managin Agent steps => add in call_model

In [ ]:
def call_model(state):
    messages = state["messages"][-5:]
    response = model.invoke(messages)
    return {"messages" : [response]}

## Force-calling a tool

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()  # Tải API key từ file .env
from pydantic import BaseModel, Field

os.environ["LANGCHAIN_TRACING_V2"] = "true"

from langchain_community.tools.tavily_search import TavilySearchResults
tools = [
    TavilySearchResults(max_results=3)]
from langgraph.prebuilt import ToolNode
tool_executor = ToolNode(tools)

from langchain_groq import ChatGroq

model = ChatGroq(model_name="llama3-70b-8192",
     temperature=0,api_key=os.getenv("GROQ_API_KEY") 
     )

from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function


functions = [format_tool_to_openai_function(t) for t in tools]

model = model.bind_functions(functions)

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    

from langchain_core.agents import AgentAction
import json
from langchain_core.messages import FunctionMessage


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

  
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    elif last_message.additional_kwargs["tool_calls"][0]["function"]["name"] == "Response":
        return "end"
    else:
        return "continue"

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages" : [response]}



def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    tool_call = last_message.additional_kwargs["tool_calls"][0]  # lấy call đầu tiên
    print(last_message.additional_kwargs["tool_calls"])
    tool_name = tool_call["function"]["name"]
    tool_args = json.loads(tool_call["function"]["arguments"])

    action = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="",  # để trống nếu không cần
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=tool_name)
    return {"messages": [function_message]}

#Modification
from langchain_core.messages import AIMessage
from uuid import uuid4
def first_model(state):
    human_input = state["messages"][-1].content
    return {
        "messages":[
            AIMessage(
                content="",
                additional_kwargs={
                    "tool_calls": [
                        {
                            "id": str(uuid4()),  # ✅ bắt buộc có id
                            "type": "function",  # ✅ bắt buộc type
                            "function": {
                                "name": "tavily_search_results",  # ✅ đúng tên tool
                                "arguments": json.dumps({"query": human_input})
                            }
                        }
                    ]
                }
            )           
        ]
    }

from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)
workflow.add_node("first_agent", first_model)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)


workflow.set_entry_point("first_agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",

        "end": END
    }
)
workflow.add_edge("action", "agent")
workflow.add_edge("first_agent", "action")

app = workflow.compile()

from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node '{key}':")
        print("------------------")
        print(value)
    print("\n---\n")  

output from node 'first_agent':
------------------
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'cc0520af-0bab-46b1-8b66-6246ee00743e', 'type': 'function', 'function': {'name': 'tavily_search_results', 'arguments': '{"query": "what is the weather in sf"}'}}]}, response_metadata={}, tool_calls=[{'name': 'tavily_search_results', 'args': {'query': 'what is the weather in sf'}, 'id': 'cc0520af-0bab-46b1-8b66-6246ee00743e', 'type': 'tool_call'}])]}

---

[{'id': 'cc0520af-0bab-46b1-8b66-6246ee00743e', 'type': 'function', 'function': {'name': 'tavily_search_results', 'arguments': '{"query": "what is the weather in sf"}'}}]
output from node 'action':
------------------
{'messages': [FunctionMessage(content="{'messages': []}", additional_kwargs={}, response_metadata={}, name='tavily_search_results')]}

---

output from node 'agent':
------------------
{'messages': [AIMessage(content="It seems like the tool_calls didn't yield a useful result. In this case, I'll r